In [1]:
import os
from numpy import dot
import scipy as sp
import pandas as pd
from numpy import ones, eye, concatenate, zeros, exp

In [2]:
import warnings

from limix._data import conform_dataset, normalize_likelihood
from limix._display import session_block

class VarDec(object):
    """
    Variance decompositon through GLMMs.

    Example
    -------

    .. doctest::

        >>> from limix.vardec import VarDec
        >>> from limix.stats import multivariate_normal as mvn
        >>> from numpy import ones, eye, concatenate, zeros, exp
        >>> from numpy.random import RandomState
        >>>
        >>> random = RandomState(0)
        >>> nsamples = 20
        >>>
        >>> M = random.randn(nsamples, 2)
        >>> M = (M - M.mean(0)) / M.std(0)
        >>> M = concatenate((ones((nsamples, 1)), M), axis=1)
        >>>
        >>> K0 = random.randn(nsamples, 10)
        >>> K0 = K0 @ K0.T
        >>> K0 /= K0.diagonal().mean()
        >>> K0 += eye(nsamples) * 1e-4
        >>>
        >>> K1 = random.randn(nsamples, 10)
        >>> K1 = K1 @ K1.T
        >>> K1 /= K1.diagonal().mean()
        >>> K1 += eye(nsamples) * 1e-4
        >>>
        >>> y = M @ random.randn(3) + mvn(random, zeros(nsamples), K0)
        >>> y += mvn(random, zeros(nsamples), K1)
        >>>
        >>> vardec = VarDec(y, "normal", M)
        >>> vardec.append(K0)
        >>> vardec.append(K1)
        >>> vardec.append_iid()
        >>>
        >>> vardec.fit(verbose=False)
        >>> print(vardec) # doctest: +FLOAT_CMP
        Variance decomposition
        ======================
        <BLANKLINE>
        𝐲 ~ 𝓝(𝙼𝜶, 0.385⋅𝙺 + 1.184⋅𝙺 + 0.000⋅𝙸)
        >>> y = exp((y - y.mean()) / y.std())
        >>> vardec = VarDec(y, "poisson", M)
        >>> vardec.append(K0)
        >>> vardec.append(K1)
        >>> vardec.append_iid()
        >>>
        >>> vardec.fit(verbose=False)
        >>> print(vardec) # doctest: +FLOAT_CMP
        Variance decomposition
        ======================
        <BLANKLINE>
        𝐳 ~ 𝓝(𝙼𝜶, 0.000⋅𝙺 + 0.350⋅𝙺 + 0.000⋅𝙸) for yᵢ ~ Poisson(λᵢ=g(zᵢ)) and g(x)=eˣ
    """
    def __init__(self, y, lik="normal", M=None):
        """
        Constructor.

        Parameters
        ----------
        y : array_like
            Phenotype.
        lik : tuple, "normal", "bernoulli", "probit", "binomial", "poisson"
            Sample likelihood describing the residual distribution.
            Either a tuple or a string specifying the likelihood is required. The
            Normal, Bernoulli, Probit, and Poisson likelihoods can be selected by
            providing a string. Binomial likelihood on the other hand requires a tuple
            because of the number of trials: ``("binomial", array_like)``. Defaults to
            ``"normal"``.
        M : n×c array_like
            Covariates matrix.
        """
        from numpy import asarray
        from glimix_core.mean import LinearMean

        y = asarray(y, float)
        data = conform_dataset(y, M)
        y = data["y"]
        M = data["M"]
        self._y = y
        self._M = M
        self._lik = normalize_likelihood(lik)
        self._mean = LinearMean(asarray(M, float))
        self._covariance = []
        self._glmm = None
        self._fit = False
        self._unnamed = 0


    @property
    def effsizes(self):
        """
        Covariace effect sizes.

        Returns
        -------
        effsizes : ndarray
            Effect sizes.
        """
        if not self._fit:
            self.fit()
        return self._mean.effsizes

    @property
    def covariance(self):
        """
        Get the covariance matrices.

        Returns
        -------
        covariances : list
            Covariance matrices.
        """
        return self._covariance
    def fit(self, verbose=True):
        """
        Fit the model.

        Parameters
        ----------
        verbose : bool, optional
            Set ``False`` to silence it. Defaults to ``True``.
        """
        with session_block("Variance decomposition", disable=not verbose):
            if self._lik[0] == "normal":
                if self._simple_model():
                    self._fit_lmm_simple_model(verbose)
                else:
                    self._fit_lmm(verbose)
            else:
                if self._simple_model():
                    self._fit_glmm_simple_model(verbose)
                else:
                    self._fit_glmm(verbose)

            if verbose:
                print(self)

        self._fit = True
    
    def lml(self):
        """
        Get the log of the marginal likelihood.

        Returns
        -------
        float
            Log of the marginal likelihood.
        """
        if not self._fit:
            self._glmm.fit()
        return self._glmm.lml()
    
    def append_iid(self, name="residual"):
        from glimix_core.cov import EyeCov

        c = EyeCov(self._y.shape[0])
        c.name = name
        self._covariance.append(c)
        
    def append(self, K, name=None):
        from numpy_sugar import is_all_finite
        from numpy import asarray
        from glimix_core.cov import GivenCov

        data = conform_dataset(self._y.values, K=K)
        K = asarray(data["K"], float)

        if not is_all_finite(K):
            raise ValueError("Covariance-matrix values must be finite.")

        K = K / K.diagonal().mean()
        cov = GivenCov(K)
        if name is None:
            name = "unnamed-{}".format(self._unnamed)
            self._unnamed += 1
        cov.name = name

        self._covariance.append(cov)

    def plot(self):
        import limix
        import seaborn as sns
        from matplotlib.ticker import FormatStrFormatter

        variances = [c.scale for c in self._covariance]
        variances = [(v / sum(variances)) * 100 for v in variances]
        names = [c.name for c in self._covariance]

        ax = sns.barplot(x=names, y=variances)
        ax.yaxis.set_major_formatter(FormatStrFormatter("%.0f%%"))
        ax.set_xlabel("random effects")
        ax.set_ylabel("explained variance")
        ax.set_title("Variance decomposition")
        with warnings.catch_warnings():
            warnings.simplefilter("ignore")
            limix.plot.get_pyplot().tight_layout()
        limix.plot.show()

    def _fit_lmm(self, verbose):
        from glimix_core.cov import SumCov
        from glimix_core.gp import GP
        from numpy import asarray

        y = asarray(self._y, float).ravel()
        gp = GP(y, self._mean, SumCov(self._covariance))
        gp.fit(verbose=verbose)
        self._glmm = gp

    def _fit_glmm(self, verbose):
        from glimix_core.cov import SumCov
        from glimix_core.ggp import ExpFamGP
        from numpy import asarray

        y = asarray(self._y, float).ravel()
        gp = ExpFamGP(y, self._lik, self._mean, SumCov(self._covariance))
        gp.fit(verbose=verbose)
        self._glmm = gp

    def _fit_lmm_simple_model(self, verbose):
        from numpy_sugar.linalg import economic_qs
        from glimix_core.lmm import LMM
        from numpy import asarray

        K = self._get_matrix_simple_model()

        y = asarray(self._y, float).ravel()
        QS = None
        if K is not None:
            QS = economic_qs(K)
        lmm = LMM(y, self._M, QS)
        lmm.fit(verbose=verbose)
        self._set_simple_model_variances(lmm.v0, lmm.v1)
        self._glmm = lmm

    def _fit_glmm_simple_model(self, verbose):
        from numpy_sugar.linalg import economic_qs
        from glimix_core.glmm import GLMMExpFam
        from numpy import asarray

        K = self._get_matrix_simple_model()

        y = asarray(self._y, float).ravel()
        QS = None
        if K is not None:
            QS = economic_qs(K)

        glmm = GLMMExpFam(y, self._lik, self._M, QS)
        glmm.fit(verbose=verbose)

        self._set_simple_model_variances(glmm.v0, glmm.v1)
        self._glmm = glmm

    def _set_simple_model_variances(self, v0, v1):
        from glimix_core.cov import GivenCov, EyeCov

        for c in self._covariance:
            if isinstance(c, GivenCov):
                c.scale = v0
            elif isinstance(c, EyeCov):
                c.scale = v1

    def _get_matrix_simple_model(self):
        from glimix_core.cov import GivenCov

        K = None
        for i in range(len(self._covariance)):
            if isinstance(self._covariance[i], GivenCov):
                self._covariance[i].scale = 1.0
                K = self._covariance[i].value()
                break
        return K

    def _simple_model(self):
        from glimix_core.cov import GivenCov, EyeCov

        if len(self._covariance) > 2:
            return False

        c = self._covariance
        if len(c) == 1 and isinstance(c[0], EyeCov):
            return True

        if isinstance(c[0], GivenCov) and isinstance(c[1], EyeCov):
            return True

        if isinstance(c[1], GivenCov) and isinstance(c[0], EyeCov):
            return True

        return False

    def __repr__(self):
        from glimix_core.cov import GivenCov
        from limix.qtl._result._draw import draw_model
        from limix._display._draw import draw_title
        covariance = ""
        for c in self._covariance:
            s = c.scale
            if isinstance(c, GivenCov):
                covariance += f"{s:.3f}⋅𝙺 + "
            else:
                covariance += f"{s:.3f}⋅𝙸 + "
        if len(covariance) > 2:
            covariance = covariance[:-3]

        msg = draw_title("Variance decomposition")
        msg += draw_model(self._lik[0], "𝙼𝜶", covariance)
        msg = msg.rstrip()

        return msg 

In [3]:
# Read in the count data
datDir = "/work-zfs/abattle4/prashanthi/project/sc_endo_diff/"
countsFile = datDir + "counts.tsv"
data_iterator = pd.read_csv(countsFile, iterator = True, chunksize=1000, sep = "\t")
counts = pd.concat([chunk for chunk in data_iterator])
counts.head()

,21843_1#10,21843_1#100,21843_1#101,21843_1#102,21843_1#103,21843_1#105,21843_1#106,21843_1#107,21843_1#108,21843_1#109,...,24539_8#88,24539_8#89,24539_8#90,24539_8#91,24539_8#92,24539_8#93,24539_8#94,24539_8#95,24539_8#97,24539_8#98
ENSG00000000003_TSPAN6,5.520777,6.456208,5.878671,4.860824,5.903640,4.513537,6.401983,5.909216,5.366645,3.228852,...,5.841814,6.104105,6.275649,7.029407,5.806978,6.199875,7.014180,6.228476,6.217161,6.034232
ENSG00000000419_DPM1,5.392461,6.065923,6.838769,6.614268,6.512403,5.527439,6.525591,6.381135,6.157296,6.248478,...,6.543807,6.369119,7.185421,6.337047,6.162437,5.885993,7.431358,7.013124,4.851771,4.937248
ENSG00000000457_SCYL3,0.000174,0.352597,0.000000,0.825955,2.201697,0.262446,0.000000,1.506837,0.283516,3.241977,...,0.017386,0.949668,0.035526,0.032044,1.773369,0.000000,0.108025,1.756339,2.492943,1.363441
ENSG00000000460_C1orf112,1.471928,4.536968,4.318528,5.373009,4.636175,4.225468,0.409785,3.668277,3.057933,3.154891,...,4.720967,3.791536,2.696476,4.227515,4.243689,3.227508,2.621121,3.950978,3.926914,4.211904
ENSG00000001036_FUCA2,2.908802,3.867327,3.321747,3.736476,4.917576,2.456866,0.577839,4.777404,2.873857,2.536708,...,3.070608,4.491643,4.206249,3.695005,2.652845,4.221847,3.188030,4.741496,3.872743,4.374577


In [4]:
# Read in the meta data
datDir = "/work-zfs/abattle4/prashanthi/project/sc_endo_diff/"
metaDataFile = datDir + "cell_metadata_cols.tsv"
metaData = pd.read_csv(metaDataFile, sep='\t')
print(len(metaData))
metaData.head()

36044


,assigned,auxDir,cell_filter,cell_name,compatible_fragment_ratio,day,donor,expected_format,experiment,frag_dist_length,...,donor_short_id,donor_long_id,pseudo,PC1_top100hvgs,PC1_top200hvgs,PC1_top500hvgs,PC1_top1000hvgs,PC1_top2000hvgs,princ_curve,princ_curve_scaled01
21843_1#10,1,aux_info,True,21843_1#10,0.999497,day1,joxm,IU,expt_09,1001,...,joxm_1,HPSI0114i-joxm_1,0.292682,-13.353833,-12.969161,-11.769526,-12.153335,-12.871002,39.972588,0.352570
21843_1#100,1,aux_info,True,21843_1#100,0.999456,day1,fafq,IU,expt_09,1001,...,fafq_1,HPSI0314i-fafq_1,0.484716,2.399795,4.633188,5.131531,7.883242,9.916888,56.433387,0.497759
21843_1#101,1,aux_info,True,21843_1#101,0.999549,day1,fafq,IU,expt_09,1001,...,fafq_1,HPSI0314i-fafq_1,0.403809,-0.612621,0.006692,-0.643021,0.185298,-0.425978,52.966142,0.467177
21843_1#102,1,aux_info,True,21843_1#102,0.999422,day1,wuye,IU,expt_09,1001,...,wuye_2,HPSI1013i-wuye_2,0.260772,-11.946009,-12.691233,-14.508021,-15.328236,-18.071351,38.419984,0.338876
21843_1#103,1,aux_info,True,21843_1#103,0.999446,day1,joxm,IU,expt_09,1001,...,joxm_1,HPSI0114i-joxm_1,0.355366,-5.735735,-5.389405,-5.802985,-6.625860,-9.430127,48.488060,0.427679


In [185]:
# Use LIMIX to perform variance component analysis
# Subsample 5000 cells to begin 
metaData_subset = metaData.sample(n = 5000)
counts_subset = counts.loc[:, metaData_subset['cell_name']]
day = pd.get_dummies(metaData_subset.day)
experiment = pd.get_dummies(metaData_subset.experiment)
donor_long_id = pd.get_dummies(metaData_subset.donor_long_id)

In [186]:
metaData_subset.head()

,assigned,auxDir,cell_filter,cell_name,compatible_fragment_ratio,day,donor,expected_format,experiment,frag_dist_length,...,donor_short_id,donor_long_id,pseudo,PC1_top100hvgs,PC1_top200hvgs,PC1_top500hvgs,PC1_top1000hvgs,PC1_top2000hvgs,princ_curve,princ_curve_scaled01
24539_8#254,1,aux_info,True,24539_8#254,0.999711,day1,zagm,IU,expt_35,1001,...,zagm_1,HPSI1013i-zagm_1,0.316911,-6.242095,-5.269772,-7.908039,-10.379629,-13.839260,46.840131,0.413144
24722_5#268,1,aux_info,True,24722_5#268,0.999563,day2,qaqx,IU,expt_38,1001,...,qaqx_1,HPSI0314i-qaqx_1,0.778171,23.130954,25.572627,31.737465,36.908752,42.122676,81.491514,0.718779
24539_3#320,1,aux_info,True,24539_3#320,0.999621,day2,guss,IU,expt_34,1001,...,guss_1,HPSI0813i-guss_1,0.747183,13.427654,16.745594,24.528007,29.165135,34.842106,75.364735,0.664739
24327_6#64,1,aux_info,True,24327_6#64,0.999653,day0,letw,IU,expt_28,1001,...,letw_1,HPSI0514i-letw_1,0.116474,-20.489037,-23.665640,-29.507710,-33.608043,-40.273055,7.781209,0.068633
21965_8#343,1,aux_info,True,21965_8#343,0.999923,day2,walu,IU,expt_24,1001,...,walu_1,HPSI0414i-walu_1,0.697490,13.593783,16.388816,18.912946,20.456644,22.747981,77.660018,0.684984


In [5]:
counts_subset = counts_subset.T
counts_subset = counts_subset.iloc[:,5]
counts_subset.head()

NameError: name 'counts_subset' is not defined

In [ ]:
day

In [189]:
experiment

,expt_09,expt_10,expt_12,expt_18,expt_20,expt_21,expt_22,expt_23,expt_24,expt_27,...,expt_36,expt_37,expt_38,expt_39,expt_40,expt_41,expt_42,expt_43,expt_44,expt_45
24539_8#254,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
24722_5#268,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
24539_3#320,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
24327_6#64,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
21965_8#343,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22194_6#299,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
24086_8#262,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
24722_7#136,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
25475_4#154,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0


In [190]:
donor_long_id

,HPSI0114i-bezi_1,HPSI0114i-eipl_1,HPSI0114i-iisa_1,HPSI0114i-iisa_3,HPSI0114i-joxm_1,HPSI0114i-kolf_2,HPSI0114i-vass_1,HPSI0114i-wegi_1,HPSI0114i-zapk_3,HPSI0115i-aoxv_3,...,HPSI1014i-sehl_6,HPSI1014i-toss_3,HPSI1014i-vils_1,HPSI1113i-bima_1,HPSI1113i-hajc_1,HPSI1113i-hayt_1,HPSI1113i-qorq_2,HPSI1113i-wahn_1,HPSI1213i-pahc_4,HPSI1213i-tolg_6
24539_8#254,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
24722_5#268,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
24539_3#320,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
24327_6#64,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
21965_8#343,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22194_6#299,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
24086_8#262,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
24722_7#136,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
25475_4#154,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [191]:
K_experiment = dot(experiment, experiment.T)
K_day = dot(day, day.T)
K_donor = dot(donor_long_id, donor_long_id.T)

In [192]:
print(len(counts_subset))
print(len(K_experiment))
print(len(K_day))
print(len(K_donor))

5000
5000
5000
5000


In [195]:
vardec = VarDec(counts_subset, "normal")
vardec.append(K_day, "day")
vardec.append(K_experiment, "exp")
vardec.append(K_donor, "donor_id")
vardec.append_iid("noise")
vardec.fit(verbose=True)

============================ Variance decomposition starts =============================

Variance decomposition
----------------------
𝐲 ~ 𝓝(𝙼𝜶, 0.015⋅𝙺 + 0.030⋅𝙺 + 0.012⋅𝙺 + 2.442⋅𝙸)


==================== Variance decomposition ends in 1452.09 seconds ====================

In [194]:
variances = [c.scale for c in vardec.covariance]
variances = [(v / sum(variances)) * 100 for v in variances]
names = [c.name for c in vardec.covariance]
print(names)
print(variances)

['exp', 'day', 'donor_id', 'noise']
[1.2043892776470868, 0.5909572629193455, 0.4611742274867033, 97.74347923194686]
